## Exploring Linkedin Summaries using Topic Modeling


In [2]:
import pandas as pd
import sklearn
import numpy as np
import matplotlib as matplot
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', -1)
%matplotlib inline
import matplotlib
plt.style.use("ggplot")
matplotlib.rcParams.update({"font.size": 14})

In [9]:
#Load the Google Data Scientist Linkedin Data
googleDatasci = pd.read_csv("Datasets/googledatasci.csv")

googleDatasci['summary'] = googleDatasci['summary'].str.strip()

#Remove the first column
del googleDatasci['Unnamed: 0']

#Select only the rows with Google as company
googleDatasci = googleDatasci.loc[(googleDatasci.company == 'Google')]

#Remove any row that doesn't contain data scientist in the headline
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Technical Program Manager") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Supply chain & Operations leader") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("DevIntel on Developer Products") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("China Display Product Lead") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Sales Operations Senior") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("APM Intern") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Trust and Safety manager") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("User Growth, Google Play") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Ads Solutions Consultant") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Regional Customer Engineer") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Software Engineering Intern") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Product Manager") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Software Developer") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Software Engineer") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Software engineer") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Opportunity Seeker") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Student at University of Toronto") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Google Policy Fellow") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Science Advocate") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Engineer") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Account Manager Display") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Customer Engineering Manager") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Strategist") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Program Manager") == False]

#print out the dataset
googleDatasci

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [ ]:
#Remove any unnecessary characters 
googleDatasci['summary'] = googleDatasci['summary'].replace(r'\t', '', regex=True) 
googleDatasci['summary'] = googleDatasci['summary'].replace(r'\n', ' ', regex=True) 
googleDatasci['summary'] = googleDatasci['summary'].replace(r'\...', ' ', regex=True)
googleDatasci['summary'] = googleDatasci['summary'].replace(r'\..', ' ', regex=True)
googleDatasci['summary'] = googleDatasci['summary'].replace('●', '', regex=True) 
googleDatasci['summary'] = googleDatasci['summary'].replace('•', '', regex=True)
googleDatasci['summary'] = googleDatasci['summary'].replace(r'\-', '', regex=True) 
googleDatasci['summary'] = googleDatasci['summary'].replace(r'\&', '', regex=True)
googleDatasci['summary'] = googleDatasci['summary'].replace(r'\$', '', regex=True)
googleDatasci['summary'] = googleDatasci['summary'].replace(',', '', regex=True)
googleDatasci['summary'] = googleDatasci['summary'].replace('\d+', '', regex=True)
googleDatasci['summary'] = googleDatasci['summary'].replace(r'\(','', regex=True)
googleDatasci['summary'] = googleDatasci['summary'].replace(r'\)','', regex=True)
googleDatasci['summary'] = googleDatasci['summary'].replace(r'\/','', regex=True)
googleDatasci['summary'] = googleDatasci['summary'].replace(r'\:','', regex=True) 
googleDatasci['summary'] = googleDatasci['summary'].replace(r'\–','', regex=True)
googleDatasci['summary'] = googleDatasci['summary'].replace(r'\%','', regex=True)
googleDatasci['summary'] = googleDatasci['summary'].replace(r'\*','', regex=True)
googleDatasci['summary'] = googleDatasci['summary'].replace(r'\@','', regex=True)
googleDatasci['summary'] = googleDatasci['summary'].replace(r'\_________','', regex=True)
googleDatasci['summary'] = googleDatasci['summary'].replace(r'\____','', regex=True)
googleDatasci['summary'] = googleDatasci['summary'].replace(r'\!','', regex=True)
googleDatasci['summary'] = googleDatasci['summary'].replace(r'\®','', regex=True)
googleDatasci['summary'] = googleDatasci['summary'].replace(r'\>','', regex=True)
googleDatasci['summary'] = googleDatasci['summary'].replace(r'\?','', regex=True)
googleDatasci['summary'] = googleDatasci['summary'].replace(r'\;','', regex=True)
googleDatasci['summary'] = googleDatasci['summary'].replace(r'\=','', regex=True)

#googleDatasci = googleDatasci.translate(None, "(){}<>")

#
googleDatasci['summary'] = googleDatasci['summary'].str.lower()

#Remove any row with a NaN
#googleDatasci['summary'] = googleDatasci['summary'].dropna()
googleDatasci = googleDatasci[googleDatasci.summary.str.contains("NaN") == False]


googleDatasci

#gs = googleDatasci['summary']

#googleSummary = pd.DataFrame(gs)
#googleSummary

In [ ]:
from nltk.tokenize import WhitespaceTokenizer
ws_tokenizer = WhitespaceTokenizer()

article = googleDatasci['summary'][1]

# tokenize example document
nyt_ws_tokens = ws_tokenizer.tokenize(article.lower())

#print (nyt_ws_tokens)

import nltk
from nltk.corpus import stopwords

## Term Weighting using TF-IDF

In [ ]:
googleDatasci['summary'].to_csv('summary.txt', index=False, sep=' ', header=None)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# we can pass in the same preprocessing parameters
stop_words = set(stopwords.words('english'))

raw_documents = []
snippets = []
with open("summary.txt","r") as fin:
    for line in fin.readlines():
        text = line.strip()
        raw_documents.append( text )
        # keep a short snippet of up to 100 characters as a title for each article
        snippets.append( text[0:min(len(text),100)] )
print("Read %d raw text documents" % len(raw_documents))

In [ ]:
vectorizer = TfidfVectorizer(stop_words, max_df = 20)
A = vectorizer.fit_transform(raw_documents)
print( "Created %d X %d TF-IDF-normalized document-term matrix" % (A.shape[0], A.shape[1]))

In [ ]:
# extract the resulting vocabulary
terms = vectorizer.get_feature_names()
print("Vocabulary has %d distinct terms" % len(terms))

In [ ]:
import operator
def rank_terms( A, terms ):
    # get the sums over each column
    sums = A.sum(axis=0)
    # map weights to the terms
    weights = {}
    for col, term in enumerate(terms):
        weights[term] = sums[0,col]
    # rank the terms by their weight over all documents
    return sorted(weights.items(), key=operator.itemgetter(1), reverse=True)

In [ ]:
ranking = rank_terms( A, terms )
for i, pair in enumerate( ranking[0:20] ):
    print( "%02d. %s (%.2f)" % ( i+1, pair[0], pair[1] ) )

## Initial Topic Modeling 

In [ ]:
# Input parameter for the number of topics to make
k = 4

In [ ]:
# create the model
from sklearn import decomposition
model = decomposition.NMF( init="nndsvd", n_components=k ) 
# apply the model and extract the two factor matrices
W = model.fit_transform( A )
H = model.components_

In [ ]:
#See the output of the model
W.shape

In [ ]:
# round to 2 decimal places for display purposes# round 
W[0,:].round(2)

In [ ]:
#See the H factor
H.shape

In [ ]:
#What is science associated with
term_index = terms.index('learning')
# round to 2 decimal places for display purposes
H[:,term_index].round(2)

## Initial Topic Descriptors

In [ ]:
def get_descriptor( terms, H, topic_index, top ):
    # reverse sort the values to sort the indices
    top_indices = np.argsort( H[topic_index,:] )[::-1]
    # now get the terms corresponding to the top-ranked indices
    top_terms = []
    for term_index in top_indices[0:top]:
        top_terms.append( terms[term_index] )
    return top_terms

In [ ]:
descriptors = []
for topic_index in range(k):
    descriptors.append( get_descriptor( terms, H, topic_index, 10 ) )
    str_descriptor = ", ".join( descriptors[topic_index] )
    print("Topic %02d: %s" % ( topic_index+1, str_descriptor ) )

In [ ]:
def plot_top_term_weights( terms, H, topic_index, top ):
    # get the top terms and their weights
    top_indices = np.argsort( H[topic_index,:] )[::-1]
    top_terms = []
    top_weights = []
    for term_index in top_indices[0:top]:
        top_terms.append( terms[term_index] )
        top_weights.append( H[topic_index,term_index] )
    # note we reverse the ordering for the plot
    top_terms.reverse()
    top_weights.reverse()
    # create the plot
    fig = plt.figure(figsize=(13,8))
    # add the horizontal bar chart
    ypos = np.arange(top)
    ax = plt.barh(ypos, top_weights, align="center", color="green",tick_label=top_terms)
    plt.xlabel("Term Weight",fontsize=14)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_top_term_weights( terms, H, 0, 10)

In [ ]:
def get_top_snippets( all_snippets, W, topic_index, top ):
    # reverse sort the values to sort the indices
    top_indices = np.argsort( W[:,topic_index] )[::-1]
    # now get the snippets corresponding to the top-ranked indices
    top_snippets = []
    for doc_index in top_indices[0:top]:
        top_snippets.append( all_snippets[doc_index] )
    return top_snippets

In [ ]:
topic_snippets = get_top_snippets( snippets, W, 1, 10 )
for i, snippet in enumerate(topic_snippets):
    print("%02d. %s" % ( (i+1), snippet ) )

In [ ]:
topic_snippets = get_top_snippets( snippets, W, 2, 10 )
for i, snippet in enumerate(topic_snippets):
    print("%02d. %s" % ( (i+1), snippet ) )

In [ ]:
from sklearn.externals import joblib
joblib.dump((W,H,terms,snippets), "articles-model-nmf-k%02d.pkl" % k)

## Advanced Topic Modeling 

In [ ]:
#Initial range of topic models
kmin, kmax = 4, 10

In [ ]:
topic_models = []
# try each value of k
for k in range(kmin,kmax+1):
    print("Applying NMF for k=%d ..." % k )
    # run NMF
    model = decomposition.NMF( init="nndsvd", n_components=k ) 
    W = model.fit_transform( A )
    H = model.components_    
    # store for later
    topic_models.append( (k,W,H) )

In [ ]:
import re
class TokenGenerator:
    def __init__( self, documents, stopwords ):
        self.documents = documents
        self.stopwords = stopwords
        self.tokenizer = re.compile( r"(?u)\b\w\w+\b" )

    def __iter__( self ):
        print("Building Word2Vec model ...")
        for doc in self.documents:
            tokens = []
            for tok in self.tokenizer.findall( doc ):
                if tok in self.stopwords:
                    tokens.append( "<stopword>" )
                elif len(tok) >= 2:
                    tokens.append( tok )
            yield tokens

In [ ]:
import gensim
docgen = TokenGenerator(raw_documents, stop_words)
# the model has 500 dimensions, the minimum document-term frequency is 20
w2v_model = gensim.models.Word2Vec(docgen, size=1, min_count=1, sg=1)

In [ ]:
print( "Model has %d terms" % len(w2v_model.wv.vocab) )

In [ ]:
w2v_model.save("w2v-model.bin")

In [ ]:
def calculate_coherence( w2v_model, term_rankings ):
    overall_coherence = 0.0
    for topic_index in range(len(term_rankings)):
        # check each pair of terms
        pair_scores = []
        for pair in combinations( term_rankings[topic_index], 2 ):
            pair_scores.append( w2v_model.similarity(pair[0], pair[1]) )
        # get the mean for all pairs in this topic
        topic_score = sum(pair_scores) / len(pair_scores)
        overall_coherence += topic_score
    # get the mean score across all topics
    return overall_coherence / len(term_rankings)

In [ ]:
def get_descriptor( all_terms, H, topic_index, top ):
    # reverse sort the values to sort the indices
    top_indices = np.argsort( H[topic_index,:] )[::-1]
    # now get the terms corresponding to the top-ranked indices
    top_terms = []
    for term_index in top_indices[0:top]:
        top_terms.append( all_terms[term_index] )
    return top_terms

In [ ]:
from itertools import combinations
k_values = []
coherences = []
for (k,W,H) in topic_models:
    # Get all of the topic descriptors - the term_rankings, based on top 10 terms
    term_rankings = []
    for topic_index in range(k):
        term_rankings.append( get_descriptor( terms, H, topic_index, 10 ) )
    # Now calculate the coherence based on our Word2vec model
    k_values.append( k )
    coherences.append( calculate_coherence( w2v_model, term_rankings ) )
    print("K=%02d: Coherence=%.4f" % ( k, coherences[-1] ) )

In [ ]:
#Select only the rows with Google as company
googleDatasci = googleDatasci.loc[(googleDatasci.company == 'Google')]

#Remove any row that doesn't contain data scientist in the headline
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Technical Program Manager") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Supply chain & Operations leader") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("DevIntel on Developer Products") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("China Display Product Lead") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Sales Operations Senior") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("APM Intern") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Trust and Safety manager") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("User Growth, Google Play") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Ads Solutions Consultant") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Regional Customer Engineer") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Software Engineering Intern") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Product Manager") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Software Developer") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Software Engineer") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Software engineer") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Opportunity Seeker") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Student at University of Toronto") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Google Policy Fellow") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Science Advocate") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Engineer") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Account Manager Display") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Customer Engineering Manager") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Strategist") == False]
googleDatasci = googleDatasci[googleDatasci.headline.str.contains("Program Manager") == False]